In [4]:
import csv
from web3 import Web3

In [5]:
# Alchemy API endpoint
alchemy_url = "https://eth-mainnet.g.alchemy.com/v2/D3TvA93qUD44YZgQu1ouV9upn3m-uIYP"

# Connect to the Alchemy API
w3 = Web3(Web3.HTTPProvider(alchemy_url))

In [15]:
# Ethereum block number to fetch
block_number = 11473329  # Ethereum blockchain, block number 11,473,329

# Fetch the block data from Alchemy
block = w3.eth.get_block(block_number)

# Define the CSV file name
csv_file_name = f"block_{block_number}_transactions.csv"

# Write the transaction data to a CSV file
with open(csv_file_name, mode="w", newline="", encoding="utf-8") as csv_file:
    fieldnames = [
        "transaction_index",  # Position of the transaction within the block
        "transaction_hash",  # Unique identifier for the transaction
        "status",  # Indicates whether the transaction succeeded (1) or failed (0)
        "from_address",  # Ethereum address of the sender
        "to_address",  # Ethereum address of the recipient
        "transaction_type",  # Indicates the type of transaction (value transfer or smart contract interaction)
        #"token_transfers",  # Information about token transfers in the transaction
        "value_wei",  # The value of the transaction (10^18 Wei = 1 ETH)
        "contract_address",  # Address of the newly created contract (if applicable)
        "input_data",  # Potentially useful information for smart contract functions and parameters used
    ]

    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    # Extract and write transaction data
    for tx_hash in block.transactions:
        tx = w3.eth.get_transaction(tx_hash)
        tx_index = block.transactions.index(tx_hash)  # Corrected position retrieval

        # Determine the transaction status based on the receipt
        receipt = w3.eth.get_transaction_receipt(tx_hash)
        status = "1" if receipt and receipt.status == 1 else "0"

        tx_data = {
            "transaction_index": tx_index,
            "transaction_hash": tx.hash.hex(),
            "status": status,
            "from_address": tx["from"],
            "to_address": tx["to"],
            "transaction_type": "Value Transfer" if str(tx["input"]) == "b''" else "Smart Contract Interaction",
            # "token_transfers": "",  # Placeholder for token transfer information
            "value_wei": tx["value"],
            "contract_address": tx["to"] if str(tx["input"]) != "b''" else "N/A",
            "input_data": tx["input"],
        }

        writer.writerow(tx_data)

print(f"Transaction data saved to {csv_file_name}")

Transaction data saved to block_11473329_transactions.csv
